In [1]:
import math
import numpy as np
import pandas as pd
from collections import defaultdict
# Split the data into train and test sets
# importing relevant libraries
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv(r'/Users/royzalta/Downloads/archive/Books Data with Category Language and Summary/Preprocessed_data.csv')

In [3]:
df_isbn = df['isbn']
num_dict = {}
for i,j in enumerate(df_isbn.unique()):
    num_dict[i]=j

In [4]:
df_isbn = df['isbn']
num_dict = {}
for i,j in enumerate(df_isbn.unique()):
    num_dict[j]=i
df['isbn_num'] = df['isbn'].apply(lambda txt: num_dict[txt])

In [5]:
#split to train andtest while keeping each user in the train and test data
train_users, test_users = train_test_split(df, test_size = 0.3 , random_state= 42)

In [6]:
x_train = train_users.drop(columns=['rating'])
y_train = train_users["rating"]

x_test = test_users.drop(columns=['rating'])
y_test = test_users["rating"]

## Implemation of TenserFlow

In [7]:
all_users = train_users["user_id"].unique()
all_books = train_users["isbn_num"].unique()

In [8]:
import tensorflow as tf

In [9]:
# user pipeline
user_input = tf.keras.layers.Input(shape=(1,), name="user")
user_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_users)(user_input)
user_embedding = tf.keras.layers.Embedding(input_dim=len(all_users)+1, output_dim=32)(user_as_integer)

In [10]:
# book pipeline
book_input = tf.keras.layers.Input(shape=(1,), name="book")
book_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_books)(book_input)
book_embedding = tf.keras.layers.Embedding(input_dim=len(all_books)+1, output_dim=32)(book_as_integer)

In [11]:
# dot product
dot = tf.keras.layers.Dot(axes=2)([user_embedding, book_embedding])
flatten = tf.keras.layers.Flatten()(dot)

In [12]:
# model input/output definition
model = tf.keras.Model(inputs=[user_input, book_input], outputs=flatten)

In [13]:
model.compile(loss="mse", metrics=[tf.keras.metrics.MeanAbsoluteError()])

In [ ]:
model.fit(
    x={
        "user": x_train["user_id"],
        "book": x_train["isbn_num"]
    },
    y=y_train.values,
    batch_size=256,
    epochs=100,
    validation_split=0.1, # for early stopping
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=1, restore_best_weights=True)
    ],
)


Epoch 1/100
2226/2538 [=========================>....] - ETA: 6s - loss: 22.8761 - mean_absolute_error: 2.8367